In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import gudhi
import pickle
import numpy as np
import gudhi as gd
from LISM_simplicial import *
from pyvis import network as net
import networkx as nx
from helpers import plot_optim

## Load data

In [4]:
with open('activation_data_mnist/simplexTree.pickle', 'rb') as handle:
    simplex_list = pickle.load(handle)

ST = gd.SimplexTree()
initial_value = -1e10

for simplex in simplex_list:
    ST.insert(simplex, initial_value)


In [5]:
G = nx.Graph()
edge_tuples = [tuple(s) for s in simplex_list if len(s)==2]
nodes = [s[0] for s in simplex_list if len(s)==1]
G.add_edges_from(edge_tuples)
G.add_nodes_from(nodes)

In [6]:
card = G.number_of_nodes()

In [7]:
g = net.Network(notebook=True)
g.from_nx(G)
g.show('data/simplextree.html')

In [8]:
stbase = gd.SimplexTree()
for i in range(len(simplex_list)):
    stbase.insert(np.array(simplex_list[i]), -1e10)
f = open('data/adv_simplextree.txt', 'w')
for (s,_) in stbase.get_filtration():
    for v in s:
        f.write(str(v) + " ")
    f.write('\n')
f.close()

convention label : 0 normal, 1 fgsm, 2 fgsm noise, 3 cw, 4 cw noise

In [9]:
fgsm = np.load('activation_data_mnist/fgsm.npy')
fgsm_noise = np.load('activation_data_mnist/fgsm_noise.npy')
normal = np.load('activation_data_mnist/normal.npy')
cw = np.load('activation_data_mnist/cw.npy')
cw_noise = np.load('activation_data_mnist/cw_noise.npy')

## Compute distance matrix with possible data leakage

In [10]:
n_inputs = 20
features = [0,1]
data = np.zeros((n_inputs * 5, 260, 2))
labels = []
for i in range(5):
    labels += [i]*n_inputs


In [11]:
data[0:n_inputs, : ,: ] = fgsm[0:n_inputs, : , features ]
data[n_inputs:2*n_inputs, : ,: ] = fgsm_noise[0:n_inputs, : , features ]
data[2*n_inputs:3*n_inputs, : ,: ] = normal[0:n_inputs, : , features ]
data[3*n_inputs:4*n_inputs, : ,: ] = cw[0:n_inputs, : , features ]
data[4*n_inputs:5*n_inputs, : ,: ] = cw_noise[0:n_inputs, : , features ]


In [12]:
data.shape

(100, 260, 2)

In [13]:
distance_matrix = adversarial_pipeline("data/adv_simplextree.txt", data, 1)

  0%|          | 0/100 [00:00<?, ?it/s]2022-05-20 13:58:07.524844: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-20 13:58:07.562619: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9c2a1240f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-20 13:58:07.562640: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
  1%|          | 1/100 [02:53<4:46:35, 173.69s/it]

### Compute a single distance for debugging 

In [21]:
theta = np.random.rand()*np.pi/2
p_ = np.array([np.cos(theta), np.sin(theta)], dtype=np.float32).reshape(2,1)
p = tf.Variable(initial_value=p_, trainable=True, dtype = tf.float32)

m1 = tf.Variable(initial_value=data[0], trainable=False, dtype=tf.float32)
m2 = tf.Variable(initial_value=data[1], trainable=False, dtype=tf.float32)

In [22]:
model0 = SimplexTreeModel_ISM(p, m1, m2, stbase = "data/adv_simplextree.txt", dim=0, card=260)
model1 = SimplexTreeModel_ISM(p, m1, m2, stbase = "data/adv_simplextree.txt", dim=1, card=260)

dist0 = LISM_optimization(model0, True)
dist1 = LISM_optimization(model1, True)


In [24]:
dist0, dist1

(1.9023528, inf)

In [211]:
model = SimplexTreeModel_ISM(p, m1, m2, stbase = "data/adv_simplextree.txt", dim=0, card=260)

opt = LISM_optimization(model)

plot_optim(opt, True)